In [23]:
import pandas as pd  # to load and manipulate data for One-Hot-Encoding
import numpy as np  # to calculate mean and standard deviation
%matplotlib widget
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier  # to build a classification tree
from sklearn.tree import plot_tree  # to draw a classification tree
from sklearn.model_selection import train_test_split  # to split data in training and testing sets
from sklearn.model_selection import cross_val_score  # for cross validation
from sklearn.metrics import plot_confusion_matrix

## import data

In [24]:
df = pd.read_csv("processed.cleveland.data", header=None)

# or download
# df = pd.read("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data",
# header=None)

df.columns = ["age", "sex", "cp", "restbp", "chol", "fbs", "restecg", "thalach", "exang", 
                "oldpeak", "slope", "ca", "thal", "hd"]

df.head()

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


## identifying missing data

In [25]:
df.dtypes

age        float64
sex        float64
cp         float64
restbp     float64
chol       float64
fbs        float64
restecg    float64
thalach    float64
exang      float64
oldpeak    float64
slope      float64
ca          object
thal        object
hd           int64
dtype: object

In [26]:
df["ca"].unique()

array(['0.0', '3.0', '2.0', '1.0', '?'], dtype=object)

In [27]:
df["thal"].unique()

array(['6.0', '3.0', '7.0', '?'], dtype=object)

## deal with missing data
* ### not much missing data relatively speaking, so deleting the rows is just fine 

In [28]:
print("missing data: ", len(df.loc[(df["ca"] == "?") | (df["thal"] == "?")]))
print("missing data in %: ", len(df.loc[(df["ca"] == "?") | (df["thal"] == "?")]) / len(df) * 100)
df.loc[(df["ca"] == "?") | (df["thal"] == "?")]

missing data:  6
missing data in %:  0.019801980198019802


,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
87,53.0,0.0,3.0,128.0,216.0,0.0,2.0,115.0,0.0,0.0,1.0,0.0,?,0
166,52.0,1.0,3.0,138.0,223.0,0.0,0.0,169.0,0.0,0.0,1.0,?,3.0,0
192,43.0,1.0,4.0,132.0,247.0,1.0,2.0,143.0,1.0,0.1,2.0,?,7.0,1
266,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0.0,?,2
287,58.0,1.0,2.0,125.0,220.0,0.0,0.0,144.0,0.0,0.4,2.0,?,7.0,0
302,38.0,1.0,3.0,138.0,175.0,0.0,0.0,173.0,0.0,0.0,1.0,?,3.0,0


In [29]:
df_no_missing = df.loc[(df["ca"] != "?") & (df["thal"] != "?")]

# no missing data anymore
df_no_missing["ca"].unique()

array(['0.0', '3.0', '2.0', '1.0'], dtype=object)

In [30]:
df_no_missing["thal"].unique()

array(['6.0', '3.0', '7.0'], dtype=object)

## split data
* ### split in classification X and predict y data sets
* ### predict: hd for hearth disease

In [31]:
X = df_no_missing.drop("hd", axis="columns").copy()
X

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57.0,0.0,4.0,140.0,241.0,0.0,0.0,123.0,1.0,0.2,2.0,0.0,7.0
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0


In [32]:
y = df_no_missing["hd"].copy()
y

0      0
1      2
2      1
3      0
4      0
      ..
297    1
298    1
299    2
300    3
301    1
Name: hd, Length: 297, dtype: int64

## one-hot encoding
* ### sklearn does support continuos data but not categorical data
* ### trick to convert categorical data to multiple columns with binary values.<br>Otherwise machine learning models would consider values like 1 & 2 closer than 1 & 4, although numbers do not have any relation
* ### only needed if there are more than 2 categories

In [33]:
X_encoded = pd.get_dummies(X, columns=["cp", "restecg", "slope", "thal"])
# alternatively: from sklearn ColumnTransporter()

X_encoded

,age,sex,restbp,chol,fbs,thalach,exang,oldpeak,ca,cp_1.0,...,cp_4.0,restecg_0.0,restecg_1.0,restecg_2.0,slope_1.0,slope_2.0,slope_3.0,thal_3.0,thal_6.0,thal_7.0
0,63.0,1.0,145.0,233.0,1.0,150.0,0.0,2.3,0.0,1,...,0,0,0,1,0,0,1,0,1,0
1,67.0,1.0,160.0,286.0,0.0,108.0,1.0,1.5,3.0,0,...,1,0,0,1,0,1,0,1,0,0
2,67.0,1.0,120.0,229.0,0.0,129.0,1.0,2.6,2.0,0,...,1,0,0,1,0,1,0,0,0,1
3,37.0,1.0,130.0,250.0,0.0,187.0,0.0,3.5,0.0,0,...,0,1,0,0,0,0,1,1,0,0
4,41.0,0.0,130.0,204.0,0.0,172.0,0.0,1.4,0.0,0,...,0,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57.0,0.0,140.0,241.0,0.0,123.0,1.0,0.2,0.0,0,...,1,1,0,0,0,1,0,0,0,1
298,45.0,1.0,110.0,264.0,0.0,132.0,0.0,1.2,0.0,1,...,0,1,0,0,0,1,0,0,0,1
299,68.0,1.0,144.0,193.0,1.0,141.0,0.0,3.4,2.0,0,...,1,1,0,0,0,1,0,0,0,1
300,57.0,1.0,130.0,131.0,0.0,115.0,1.0,1.2,1.0,0,...,1,1,0,0,0,1,0,0,0,1


* ### y contains categorical data from 0 to 4. For simplicity data points greater than 1 will be converted to 1

In [34]:
y[y > 1] = 1
y.unique()

array([0, 1], dtype=int64)

## preliminary decision tree
* ### Note: use random_state to recreate the same model

In [43]:
# split data into training and testing data
# default: test_size=0.25 & shuffle=True 
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=42)

# create a decision tree and fit it to the training data
clf_dt = DecisionTreeClassifier(random_state=42)
clf_dt = clf_dt.fit(X_train, y_train)

# plotting the tree
plt.figure(figsize=(15, 7.5))
plot_tree(clf_dt, filled=True, rounded=True, class_names=["No HD", "Yes HD"], feature_names=X_encoded.columns)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(666.4331896551724, 553.4375, 'ca <= 0.5\ngini = 0.498\nsamples = 222\nvalue = [118, 104]\nclass = No HD'),
 Text(385.82974137931035, 505.3125, 'thal_7.0 <= 0.5\ngini = 0.382\nsamples = 132\nvalue = [98, 34]\nclass = No HD'),
 Text(235.50646551724137, 457.1875, 'oldpeak <= 2.7\ngini = 0.24\nsamples = 93\nvalue = [80, 13]\nclass = No HD'),
 Text(170.36637931034483, 409.0625, 'age <= 58.5\ngini = 0.185\nsamples = 87\nvalue = [78, 9]\nclass = No HD'),
 Text(100.2155172413793, 360.9375, 'chol <= 311.5\ngini = 0.061\nsamples = 63\nvalue = [61, 2]\nclass = No HD'),
 Text(60.12931034482759, 312.8125, 'restbp <= 109.0\ngini = 0.033\nsamples = 60\nvalue = [59, 1]\nclass = No HD'),
 Text(40.08620689655172, 264.6875, 'sex <= 0.5\ngini = 0.278\nsamples = 6\nvalue = [5, 1]\nclass = No HD'),
 Text(20.04310344827586, 216.5625, 'gini = 0.0\nsamples = 4\nvalue = [4, 0]\nclass = No HD'),
 Text(60.12931034482759, 216.5625, 'exang <= 0.5\ngini = 0.5\nsamples = 2\nvalue = [1, 1]\nclass = No HD'),
 Tex

In [36]:
# plot confusion matrix
fig, ax = plt.subplots(figsize=(7, 4))
plot_confusion_matrix(clf_dt, X_test, y_test, display_labels=["Does not have HD", "Has HD"], ax=ax)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## optimize tree: cost complexity pruning - visualize alpha

In [37]:
# determine values for alpha
path = clf_dt.cost_complexity_pruning_path(X_train, y_train)
# extract different values for alpha
ccp_alphas = path.ccp_alphas
# exclude max alpha which is the root
ccp_alphas = ccp_alphas[:-1]

# list to put in decision trees
clf_dts = []

# create 1 decision tree per value for alpha and append to list
for ccp_alpha in ccp_alphas:
    clf_dt = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf_dt.fit(X_train, y_train)
    clf_dts.append(clf_dt)

In [38]:
train_scores = [clf_dt.score(X_train, y_train) for clf_dt in clf_dts]
test_scores = [clf_dt.score(X_test, y_test) for clf_dt in clf_dts]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs. alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker="o", label="train", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker="o", label="test", drawstyle="steps-post")
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## cost complexity pruning: cross validation - finding best alpha
* ### Note: as the graph above suggest the best alpha is around 0.016 but it is not
* ### Note: grid search also possbile

In [39]:
clf_dt = DecisionTreeClassifier(ccp_alpha=0.016)
scores = cross_val_score(clf_dt, X_train, y_train, cv=5)
df = pd.DataFrame(data={"tree": range(5), "accuracy": scores})
df.plot(x="tree", y="accuracy", title="alpha = 0.016", marker="o", linestyle="--", figsize=(5, 4))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
# array to store the result of each fold during cross validation
alpha_loop_values = []

# doing 5-fold cross validation and save mean and std of scores
for ccp_alpha in ccp_alphas:
    clf_dt = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    # returns: array of scores for each run of the cross validation
    scores = cross_val_score(clf_dt, X_train, y_train, cv=5)
    alpha_loop_values.append([ccp_alpha, np.mean(scores), np.std(scores)])

# plot for each alpha mean and std of scores
alpha_results = pd.DataFrame(alpha_loop_values, columns=["alpha", "mean_accuracy", "std"])

# yerr: y error, the smaller the better
alpha_results.plot(x="alpha", y="mean_accuracy", yerr="std", marker="o", linestyle="--", figsize=(5, 4))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## evaluating alpha
* ### alpha should be close to 0.14 according to the graph above

In [41]:
ideal_ccp_alpha = alpha_results[(alpha_results["alpha"] > 0.014)
                    & (alpha_results["alpha"] < 0.015)]["alpha"]
ideal_ccp_alpha

20    0.014225
Name: alpha, dtype: float64

In [ ]:
# convert to series to float
ideal_ccp_alpha = float(ideal_ccp_alpha)
ideal_ccp_alpha

0.014224751066856332

## final classification tree

In [ ]:
# create a decision tree and fit it to the training data
clf_dt_pruned = DecisionTreeClassifier(random_state=42, ccp_alpha=ideal_ccp_alpha)
clf_dt_pruned = clf_dt_pruned.fit(X_train, y_train)

# plot confusion matrix
fig, ax = plt.subplots(figsize=(7, 4))
plot_confusion_matrix(clf_dt_pruned, X_test, y_test, display_labels=["Does not have HD", "Has HD"], ax=ax)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# plotting the tree
plt.figure(figsize=(8, 5))
plot_tree(clf_dt_pruned, filled=True, rounded=True, class_names=["No HD", "Yes HD"],
          feature_names=X_encoded.columns)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …